In [3]:
include("utils.jl")
include("replicated_AP.jl")

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly C:\Users\leope\.julia\packages\Gadfly\09PWZ\src\mapping.jl:228


_afp_compute_energy (generic function with 1 method)

# Loading Data

PATH to dataset folder - datasets can be downloaded @  https://drive.google.com/open?id=1N8MdN_x-j2wV35h6xrDaajqRGa5C1LYj

Folder Structure is in listing.txt file

In [4]:
main_folder = "C:/Users/leope/Google Drive/EPFL/Tesi/modified_AP/datasets/";

Flights

In [ ]:
folder = "/detecting_cities"

similarities = readdlm(main_folder * folder * "/similarities.csv")
coordinates = readdlm(main_folder * folder * "/coordinates.csv");

In [ ]:
S = list2matrix(similarities, [-4500.0]);

Docs, summarization

In [ ]:
folder = "/doc_summarization"

similarities = readdlm(main_folder * folder * "/DocumentSummarizationSimilarities.txt")
preferences = readdlm(main_folder *  folder * "/DocumentSummarizationPreferences.txt")[:,1];

In [ ]:
S = list2matrix(similarities, preferences);

G2

In [ ]:
folder = "/g2"
filename = "/g2-2-40"

S, true_assignments, D = eucledian_space2similarities(main_folder, folder, filename; subset_size=1024, 
        lambda=-200.0, partitioning_flag=true);

DIM-sets

In [ ]:
folder = "/DIM-sets"
filename = "/dim128"

S, D = eucledian_space2similarities(main_folder, folder, filename);

# AP, rAP single run 

Step number and messages average values are printed. 
era per controllare eventuali divergenze o messaggi che vanno a zero...

In [ ]:
@time sol_AP = affinityprop_mod(S, damp=0.8, maxiter=200, run_vanilla=true)

In [ ]:
@time sol_AP_mod = affinityprop_mod(S, damp=0.8, maxiter=200, y=10.0, γ=1.0, γfact=1.01, run_vanilla=false)

# Experiments for different datasets and values of parameters

All results are stored in a dataframe for each dataset considered in order to do plots and analysis

In [ ]:
##### Parameters #####

N = size(S)[1]

damp=0.8
y=10.0
γ=0.5
γfact=1.01

repetitions = 100

partitions_flag = true

dataframe_path = "./results_dataframes" * filename * ".csv"

# Varying parameters #

damping = [0.8]
γ_list = [(0.5,1.1), (.1,1.05), (.5,1.05), (1.0,1.05)]

# ------------------ #

if isfile(dataframe_path)
    df = CSV.read(dataframe_path)
else
    df = DataFrame(Algo = String[], Energy = Float64[], Damp = Float64[], y = Float64[], γ = Float64[], γfact = Float64[],
            D = Int[], N = Int[], Assignments = Float64[])
end

for (γ, γfact) in γ_list
    for damp in damping
        for rep in 1:repetitions
            println("repetition ", rep, "/", repetitions)
            @time sol_AP = affinityprop_mod(S, damp=damp, maxiter=200, run_vanilla=true)
            @time sol_AP_mod = affinityprop_mod(S, damp=damp, maxiter=200, y=y, γ=γ, γfact=γfact, run_vanilla=false)

            if partitions_flag
                temp1 = mean(sol_AP.assignments .== true_assignments)
                temp1 = max.(1 .- temp1, temp1)

                temp2 = mean(sol_AP_mod.assignments .== true_assignments)
                temp2 = max.(1 .- temp2, temp2)
            else
                temp1, temp2 = nothing, nothing
            end

            push!(df, ["AP", sol_AP.energy / N, damp, -1.0, -1.0, -1.0, D, N, temp1])
            push!(df, ["rAP", sol_AP_mod.energy / N, damp, y, γ, γfact, D, N, temp2])
        end
        CSV.write(dataframe_path, df);
    end
end

# Plotting

to be arranged case to case

In [10]:
filename = "/g2-2-40"
df = CSV.read("./results_dataframes" * filename * ".csv");

In [13]:
add_legend_column_to_df!(df);

In [17]:
Set(df.N)

Set(Union{Missing, Int64}[1024, 1000, 220, 200])

In [20]:
# Filter the dataframe for plotting
N = 1024
df_plot = df[(df.N .== N) ,:]; #.& (df.Damp .== 0.8)

In [21]:
pofE = Gadfly.plot(df_plot, x=:Energy, 
    color=:Legend, Geom.density(bandwidth=0.0033), #Geom.histogram(bincount=20,density=true, position=:dodge), #33 7000
    #Coord.Cartesian(xmin=.72,xmax=.74),  
    #Coord.Cartesian(xmin=.4,xmax=3),
    #Coord.Cartesian(xmin=.031,xmax=0.032),

    #Scale.y_log10,
    Guide.xlabel("Energy density (per datapoint)"), Guide.ylabel("pdf"),
    Guide.title("Energy distribution @ convergence for $filename\n
                    Sub-sample of $N points out of 2048."))
#     Guide.title(format("Energy distribution of AP after convergence for random message initialization \n 
#                 Dataset: D={:d}, N={:d} and k=16 \n
#                 Params: damp={:.1f}, y={:.1f}, γ={:.4f}, γfact={:.3f}\n
#                 Initializations are random in [-0.01,0]", D, N, damp, y, γ, γfact)),

draw(PDF(format("./figures/AP_vs_rAP_accuracy_histogram_{:d}subset_{:s}.pdf", N, replace(filename, "/" => "")), 800px, 400px), pofE)